In [11]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import vectorbt as vbt
import numpy as np

In [12]:
mt5.initialize()

True

In [13]:
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 10, 31)
timeframe = mt5.TIMEFRAME_H1

asset = "EURUSD"

price_df = mt5.copy_rates_range(asset, timeframe, start_date, end_date)
assert price_df.shape[0] > 0
price_df = pd.DataFrame(price_df)

price_df.loc[:, "time_dt"] = pd.to_datetime(price_df.time, unit="s")
price_df.loc[:, "week"] = price_df.loc[:, "time_dt"].apply(lambda date: date.isocalendar()[1])

price_df = price_df.sort_index()

o, h, l, c = (price_df['open'], price_df['high'], price_df['low'], price_df['close'])
o_1, h_1, l_1, c_1 = o.shift(1), h.shift(1), l.shift(1), c.shift(1)

# Your logic applied at t using (t) and (t-1)
bull_order_block = (c > o) & (o_1 > c_1) & (h > h_1)
bear_order_block = (c < o) & (o_1 > c_1) & (l < l_1)

cross_pivot_low = price_df.low.vbt.crossed_below(price_df.low.rolling(40).min().shift(1))
cross_pivot_low_last_n = cross_pivot_low.rolling(3).apply(lambda x: np.any(x), raw=True)
bool_ts = (cross_pivot_low_last_n.astype(bool) & bull_order_block)

short_sma = 8*24
long_sma = 50*24

short_sma_ts = price_df.close.rolling(short_sma).mean()
long_sma_ts = price_df.close.rolling(long_sma).mean()

bool_ts = bool_ts & (short_sma_ts > long_sma_ts)

# Compute TP and SL based on first 10% historical data
mean_two_hour_move = price_df.close.rolling(2).apply(lambda x: abs(x[-1]-x[0])/x[0], raw=True).describe()
mean_two_hour_move = mean_two_hour_move["mean"]

sl = mean_two_hour_move
rr = 5
tp = sl*3

pf = vbt.Portfolio.from_signals(close=price_df.close, entries=bool_ts, sl_stop=sl, tp_stop=tp, freq="h")

In [367]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import vectorbt as vbt
from datetime import datetime
import matplotlib.pyplot as plt



def load_data(symbol=SYMBOL, start=START_DATE, end=END_DATE, tf=TIMEFRAME):
    """Load OHLC data from MT5"""
    df = mt5.copy_rates_range(symbol, tf, start, end)
    assert df.shape[0] > 0, f"No data for {symbol}"
    df = pd.DataFrame(df)
    df['time_dt'] = pd.to_datetime(df.time, unit="s")
    return df.sort_index()

def add_indicators(df, sma_short=SMA_SHORT_HOURS, sma_long=SMA_LONG_HOURS, pivot_window=PIVOT_WINDOW):
    """Add technical indicators"""
    df['sma_short'] = df.close.rolling(sma_short).mean()
    df['sma_long'] = df.close.rolling(sma_long).mean()
    df['pivot_low'] = df.low.rolling(pivot_window).min().shift(1)
    df['uptrend'] = df.sma_short > df.sma_long
    return df

def get_order_blocks(df):
    """Calculate order block patterns"""
    o, h, l, c = df['open'], df['high'], df['low'], df['close']
    o_1, h_1, l_1, c_1 = o.shift(1), h.shift(1), l.shift(1), c.shift(1)

    df['bull_ob'] = (c > o) & (o_1 > c_1) & (h > h_1)
    df['bear_ob'] = (c < o) & (o_1 > c_1) & (l < l_1)
    return df

def generate_signals(df, lookback=BREAKOUT_LOOKBACK):
    """Generate entry signals"""
    # Pivot breakout
    cross_pivot = df.low.vbt.crossed_below(df.pivot_low)
    cross_recent = cross_pivot.rolling(lookback).apply(lambda x: np.any(x), raw=True).astype(bool)

    # Combine all conditions
    df['long_signal'] = cross_recent & df.bull_ob & df.uptrend
    df['short_signal'] = False  # Add short logic if needed

    return df

def calculate_stops(df, atr_period=ATR_PERIOD, sl_mult=SL_MULTIPLIER, tp_mult=TP_MULTIPLIER):
    """Calculate dynamic stops"""
    atr = df.close.rolling(atr_period).apply(
        lambda x: abs(x[-1] - x[0]) / x[0], raw=True
    ).mean()
    return atr * sl_mult, atr * tp_mult

def run_backtest(df, sl, tp, init_cash=10000, size=1000, fees=0.0002):
    """Run vectorbt backtest"""
    return vbt.Portfolio.from_signals(
        close=df.close,
        entries=df.long_signal,
        sl_stop=sl,
        tp_stop=tp,
        freq="h",
        init_cash=init_cash,
        size=size,
        fees=fees
    )

def plot_results(df, pf):
    """Quick visualization"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

    # Price and signals
    ax1.plot(df.index, df.close, 'b-', alpha=0.6, label='Close')
    ax1.plot(df.index, df.sma_short, 'g--', alpha=0.5, label=f'SMA {SMA_SHORT_HOURS}h')
    ax1.plot(df.index, df.sma_long, 'r--', alpha=0.5, label=f'SMA {SMA_LONG_HOURS}h')
    ax1.scatter(df.index[df.long_signal], df.close[df.long_signal],
                marker='^', c='lime', s=100, zorder=5, label='Long Signal')
    ax1.legend(loc='upper left')
    ax1.set_ylabel('Price')
    ax1.grid(True, alpha=0.3)

    # Portfolio value
    ax2.plot(pf.value(), 'k-', label='Portfolio Value')
    ax2.fill_between(pf.value().index, pf.value().values, alpha=0.3)
    ax2.set_ylabel('Portfolio Value')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
    ax2.legend()

    plt.tight_layout()
    plt.show()

def print_stats(pf):
    """Print performance metrics"""
    print(f"Total Return: {pf.total_return():.2%}")
    print(f"Sharpe Ratio: {pf.sharpe_ratio():.2f}")
    print(f"Max Drawdown: {pf.max_drawdown():.2%}")
    print(f"Win Rate: {pf.win_rate():.2%}")
    print(f"Total Trades: {pf.total_trades()}")
    print(f"Profit Factor: {pf.profit_factor():.2f}")

# Main pipeline
def run_strategy():
    """Run complete strategy pipeline"""
    # Load and prepare data


    # Calculate stops
    sl, tp = calculate_stops(df)

    # Run backtest
    pf = run_backtest(df, sl, tp)

    # Results
    print_stats(pf)
    plot_results(df, pf)

    return df, pf


In [370]:
# Backtest parameters
SYMBOL = "EURUSD"
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 10, 31)
TIMEFRAME = mt5.TIMEFRAME_H1

# Strategy parameters
PIVOT_WINDOW = 40
BREAKOUT_LOOKBACK = 3
SMA_SHORT_HOURS = 8 * 24
SMA_LONG_HOURS = 50 * 24

# Risk parameters
ATR_PERIOD = 2
SL_MULTIPLIER = 1.0
TP_MULTIPLIER = 3.0

In [371]:
df = load_data()
df = add_indicators(df)
df = get_order_blocks(df)
df = generate_signals(df)

vbt.Portfolio.from_signals(
        close=df.close,
        entries=df.long_signal,
        sl_stop=sl,
        tp_stop=tp,
        freq="h",
        init_cash=100000,
    )

In [342]:
import vectorbt as vbt

In [360]:
pf.stats()

Start                                                 0
End                                               11423
Period                                476 days 00:00:00
Start Value                                       100.0
End Value                                    103.248425
Total Return [%]                               3.248425
Benchmark Return [%]                           4.756687
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               1.319414
Max Drawdown Duration                 182 days 23:00:00
Total Trades                                         62
Total Closed Trades                                  62
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  41.935484
Best Trade [%]                                 1.167182
Worst Trade [%]                               -0

In [361]:
pf.plot_trades()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c204aebd-b4f8-4176-a07b-5bd74277f2d8',
              'x': array([    0,     1,     2, ..., 11421, 11422, 11423], dtype=int64),
              'y': array([1.10434, 1.10364, 1.10381, ..., 1.15642, 1.15695, 1.15687])},
             {'customdata': array([[0, 0, 91.8189330639978, 0.0, 'Long'],
                                   [1, 1, 92.16786448882249, 0.0, 'Long'],
                                   [2, 2, 92.27777578550895, 0.0, 'Long'],
                                   ...,
                                   [59, 59, 88.00140789109798, 0.0, 'Long'],
                                   [60, 60, 88.43559799346771, 0.0, 'Long'],
                                   [61, 61, 88.911453057241, 0.0, 'Long']], dtype=object),
              'hovertemplate': ('Exit Trade Id: %{customdata[0]' ... 'br>Direction: %{customdata[

In [362]:
pf.plot_cum_returns()

FigureWidget({
    'data': [{'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': '46daf0ec-39fd-4d49-8c39-7c286197b2f5',
              'x': array([    0,     1,     2, ..., 11421, 11422, 11423], dtype=int64),
              'y': array([1.        , 0.99936614, 0.99952008, ..., 1.04715939, 1.04763931,
                          1.04756687])},
             {'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '65f668dc-a423-40fd-8afe-770278186f84',
              'x': array([    0,     1,     2, ..., 11421, 11422, 11423], dtype=int64),
              'y': array([1, 1, 1, ..., 1, 1, 1])},
             {'connectgaps': False,
              'fill': 'tonexty',
              'fillcolor': 'rgba(0, 128, 0, 0.3)',
              'hoverinfo': 'skip',
      

In [1]:
from metalib.metaob import MetaOrderBlock

In [15]:
test = MetaOrderBlock(["EURUSD"], mt5.TIMEFRAME_H1, "test_ob", 0.01)

In [16]:
test.fit()

test_ob:: No model to fit for MetaOrderBlock


In [19]:
start_date = datetime(2025, 9, 1)
end_date = datetime(
test.signals()

KeyError: 'EURUSD'

In [21]:
from datetime import datetime, timedelta
import pytz

In [42]:
end_time = datetime.now(pytz.utc)
start_time = end_time - timedelta(days=90)

try:
    test.run(start_time, end_time)
except Exception as e:
    print(f"Error running strategy: {e}")

C:\Users\Trismegist\anaconda3\envs\adonys\lib\site-packages\tables\path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
C:\Users\Trismegist\Documents\GitHub\metalib\metalib\metastrategy.py:98: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['close', 'sma_short', 'sma_long', 'uptrend', 'pivot_low', 'pivot_high',
       'atr', 'bull_ob', 'bear_ob', 'cross_below_pivot', 'cross_above_pivot',
       'timestamp', 'long_signal', 'short_signal'],
      dtype='object')]

  store.put(day_group, updated_data)


In [43]:
test.loadData(start_time, end_time)
test.debug=True

In [ ]:
test.signals()

In [45]:
test.data

{'EURUSD':                         open     high      low    close  tick_volume  spread  \
 time                                                                           
 2025-08-11 00:00:00  1.16400  1.16476  1.16400  1.16430          345      27   
 2025-08-11 01:00:00  1.16436  1.16509  1.16422  1.16466         1023       3   
 2025-08-11 02:00:00  1.16468  1.16507  1.16444  1.16473          804       2   
 2025-08-11 03:00:00  1.16473  1.16484  1.16418  1.16474         1466       2   
 2025-08-11 04:00:00  1.16474  1.16588  1.16454  1.16588         1480       2   
 ...                      ...      ...      ...      ...          ...     ...   
 2025-11-07 19:00:00  1.15777  1.15822  1.15715  1.15803         1680       0   
 2025-11-07 20:00:00  1.15804  1.15836  1.15675  1.15683         1355       0   
 2025-11-07 21:00:00  1.15676  1.15727  1.15556  1.15596         1761       0   
 2025-11-07 22:00:00  1.15594  1.15668  1.15574  1.15643         1499       0   
 2025-11-07 23:00: